Python实现经典机器学习案例 良/恶性性乳腺癌肿瘤完整数据预测

分析思路

相比于良性肿瘤，人们更关注是否为恶性肿瘤，所以将恶性肿瘤设为正例，为了检验模型构建是否准确、样本是否均衡，引入召回率、精确率和AUC指标，AUC越接近1.0，检测方法真实性越高;等于0.5时，则真实性最低，无应用价值。

分析流程

1.获取数据； 2.数据处理（缺失值处理）； 3.数据集划分； 4.特征工程（无量纲化处理——标准化）； 5.逻辑回归预估器； 6.模型评估（F1-score&AUC)

In [2]:
import pandas as pd
import numpy as np
# 1、读取数据
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
column_name = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                   'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
                   'Normal Nucleoli', 'Mitoses', 'Class']
data = pd.read_csv(path,names=column_name)
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [3]:
data.describe()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [4]:
# 缺失值处理
#替换np.nan，根据UCI提示，数据中缺失值均用‘？’代替
data = data.replace(to_replace='?',value=np.nan)
data.isnull().sum()
data.dropna(inplace=True)
data.isnull().sum()

Sample code number             0
Clump Thickness                0
Uniformity of Cell Size        0
Uniformity of Cell Shape       0
Marginal Adhesion              0
Single Epithelial Cell Size    0
Bare Nuclei                    0
Bland Chromatin                0
Normal Nucleoli                0
Mitoses                        0
Class                          0
dtype: int64

In [5]:
data.shape

(683, 11)

分割数据集

In [6]:
from sklearn.model_selection import train_test_split
# 随机采样 25% 的数据用于测试，剩下的 75% 用于构建训练集合。
X_train,X_test,y_train,y_test = train_test_split(data[column_name[1:10]],data['Class'],test_size=0.25,random_state=33)

数据标准化

In [7]:
from sklearn.preprocessing import StandardScaler
# 标准化数据，保证每个维度的特征数据方差为 1 ，均值为 0 。使得预测结果不会被某些维度过大的特征值而主导。
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

逻辑回归预估器

In [8]:
from sklearn.linear_model import LogisticRegression
# 分别使用 Logistic 回归建立模型，代码如下：
# 初始化 LogisticRegression
lr = LogisticRegression()
# 调用 LogisticRegression 中的 fit 函数/模块用来训练模型参数
lr.fit(X_train,y_train)

LogisticRegression()

模型评估

In [10]:
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
# 使用训练好的模型 lr 对 X_test 进行预测，结果储存在变量 lr_y_predict 中。
# 然后使用 classification_report 函数来评估模型的性能，包括精确率、召回率、F1-score 等指标。
# 最后使用 confusion_matrix 函数来计算混淆矩阵，并使用 roc_auc_score 函数来计算 AUC 值。
y_pred = lr.predict(X_test)
# 使用逻辑斯蒂回归模型自带的评分函数 score 获得模型在测试集上的准确性结果
print ('Accuracy of LR Classifier:', lr.score(X_test, y_test))
print(classification_report(y_test,y_pred))
# 打印分类报告，提供多个评估指标：
# Precision（精确率）：正确预测为正的比例
# Recall（召回率）：实际为正中被正确预测的比例
# F1-score：Precision 和 Recall 的调和平均
# Support：每个类的样本数
# Accuracy（准确率）：整体正确预测的比例（只在多分类时显示）
# Macro avg：各类别指标的未加权平均
# Weighted avg：按支持数加权的平均
print(confusion_matrix(y_test,y_pred))
# 打印混淆矩阵，形式通常为：
# [ [TN FP]
#   [FN TP] ]
# TN: 真阴性，FP: 假阳性
# FN: 假阴性，TP: 真阳性
print(roc_auc_score(y_test,y_pred))
# 计算并打印 ROC AUC 分数（Area Under the ROC Curve）
# 通常应该传入预测概率（如 lr.predict_proba(X_test)[:, 1]）而非预测类别
# 如果直接传入类别预测，对于二分类会计算准确率，多分类会有特殊处理
# 值范围 0.5-1.0，越接近 1 表示模型越好

Accuracy of LR Classifier: 0.9883040935672515
              precision    recall  f1-score   support

           2       0.99      0.99      0.99       100
           4       0.99      0.99      0.99        71

    accuracy                           0.99       171
   macro avg       0.99      0.99      0.99       171
weighted avg       0.99      0.99      0.99       171

[[99  1]
 [ 1 70]]
0.9879577464788732
